# Profiling tiling of titiler-pgstac and titiler-xarray

This notebook profiles code for tiling CMIP6 data via 2 methods:

1. pgSTAC + COGs: The first method uses a (local or remote) pgSTAC database for storing metadata about COGs on S3. The libraries used are pgstac for reading STAC metadata and rio_tiler's rasterio for reading COGs on S3.
2. kerchunk + netCDF: The second method uses a (local or S3) kerchunk reference file for NetCDF files stored on S3. The libraries used are xarray for reading the Zarr metadata and rio_tiler's XarrayReader for reading data from the NetCDFs on S3.
3. Zarr stores with different chunking configurations.

In the future, the following improvements and additions to this profiling code will be made:

1. Test zarr stores with pyramids
2. Test different tiles + sum the results.
3. Test a higher resolution dataset.

# Findings: Executive summary

1. pgSTAC + COGs is fastest, assuming GDAL environment variables are configured optimally
2. tiling using Zarr really depends on the chunking structure. Make sure that:
   1. if optimizing for spatial visualization and not time series, only store 1 time step per chunk
   2. Make sure the coordinates themselves are not chunked, otherwise this will require a lot of data fetches when loading the dataset
3. Pyramids can be used to generate low resolution versions of the dataset.

## Setup / Step 0

1. Load some basic libraries
2. Set an initial tile to test
3. Add some AWS credentials
4. Install any missing libraries

In [4]:
%load_ext autoreload
%autoreload

import boto3
from datetime import datetime
import io
from PIL import Image
import os
import warnings
import cmip6_zarr.eodc_hub_role
from matplotlib.pyplot import imshow
import morecantile
import numpy as np
import pandas as pd
from profiler.main import Timer, cprofile_list_to_dict

from cmip6_zarr import eodc_hub_role
credentials = eodc_hub_role.fetch_and_set_credentials()
import cmip6_pgstac.profile_pgstac as profile_pgstac
pool = profile_pgstac.connect_to_database(credentials)

warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Inbound rule added successfully.
Connected to database


In [5]:
%%capture
!pip install morecantile==3.4.0 loguru titiler titiler-pgstac
!pip install psycopg psycopg_binary psycopg_pool

In [6]:
%%capture
!pip install -r requirements.txt

In [7]:
tms = morecantile.tms.get("WebMercatorQuad")
zooms = range(12) # Zoom 10 is the level at which you can see large roads, 15 is buildings
xyz_tiles = []
for z in zooms:
    # lat/lon over central park
    tile = tms.tile(-73.97, 40.78, z)
    xyz_tiles.append((tile.x, tile.y, tile.z))

xyz_tiles

[(0, 0, 0),
 (0, 0, 1),
 (1, 1, 2),
 (2, 3, 3),
 (4, 6, 4),
 (9, 12, 5),
 (18, 24, 6),
 (37, 48, 7),
 (75, 96, 8),
 (150, 192, 9),
 (301, 384, 10),
 (603, 769, 11)]

In [8]:
#parameters
temporal_resolution = "daily"
model = "GISS-E2-1-G"
variable = "tas"
anon=True

## Profile titiler-pgstac

To achieve the best performance, we set some GDAL environment variables.

These variables are documented here https://developmentseed.org/titiler/advanced/performance_tuning/, but that advice is copied into comments below for ease of reference.

In [ ]:
gdal_env_vars = {
    # By default GDAL reads the first 16KB of the file, then if that doesn't contain the entire metadata
    # it makes one more request for the rest of the metadata.
    # In environments where latency is relatively high, AWS S3,
    # it may be beneficial to increase this value depending on the data you expect to read.    
    'GDAL_INGESTED_BYTES_AT_OPEN': '32768',
    # It's much better to set EMPTY_DIR because this prevents GDAL from making a LIST request.
    # LIST requests are made for sidecar files, which does not apply for COGs    
    'GDAL_DISABLE_READDIR_ON_OPEN': 'EMPTY_DIR',
    # Tells GDAL to merge consecutive range GET requests.    
    'GDAL_HTTP_MERGE_CONSECUTIVE_RANGES': 'YES',
    # When set to YES, this attempts to download multiple range requests in parallel, reusing the same TCP connection. 
    # Note this is only possible when the server supports HTTP2, which many servers don't yet support.
    # There's no downside to setting YES here.    
    'GDAL_HTTP_MULTIPLEX': 'YES',
    'GDAL_HTTP_VERSION': '2',
    # Setting this to TRUE enables GDAL to use an internal caching mechanism. It's recommended (strongly).    
    'VSI_CACHE': 'TRUE'
}

def set_or_unset_gdal(set_vars=True):
    if set_vars:
        for key, value in gdal_env_vars.items():
            os.environ[key] = value
    else:
        for key, value in gdal_env_vars.items():
            os.environ.pop(key, None)
    return

In [ ]:
if temporal_resolution == 'daily':
    collection = f"CMIP6_daily_{model}_{variable}"
elif temporal_resolution == 'monthly':
    collection = f"CMIP6_ensemble_monthly_median_{variable}"

query = {
  "collections": [ collection ],
  "filter": {
    "op": "t_intersects",
    "args": [
      {
        "property": "datetime"
      },
      {
        "interval": [
          "1950-04-01T00:00:00Z"           
        ]
      }
    ]
  },
  "filter-lang": "cql2-json"
}

gdal_results_df = {}
niters = 1
xyz_tile = (0,0,0)
for settings in ['with_gdal_vars', 'without_gdal_vars']:
    gdal_results_df[settings] = { 'tile times': [], 'mean total time': None }
    results_timings = gdal_results_df[settings]
    if settings == 'with_gdal_vars':
        set_or_unset_gdal(set_vars=True)
    else:
        set_or_unset_gdal(set_vars=False)
    for iter in range(niters):
        with Timer() as t:
            image_and_assets, cprofile = profile_pgstac.tile(pool, *xyz_tile, query=dict(query))
        results_timings['tile times'].append(round(t.elapsed * 1000, 2))
    results_timings['mean total time'] = np.mean(results_timings['tile times'])
    

In [ ]:
pd.DataFrame.from_dict(cog_results_df, orient='index')

In [ ]:
# Results for different tiles
tile_results_df = {}
niters = 10
set_or_unset_gdal(set_vars=True)
for xyz_tile in xyz_tiles:
    tile_results_df[xyz_tile] = { 'tile times': [], 'mean total time': None }
    results_timings = tile_results_df[xyz_tile]
    for iter in range(niters):
        with Timer() as t:
            image_and_assets, cprofile = profile_pgstac.tile(pool, *xyz_tile, query=dict(query))
        results_timings['tile times'].append(round(t.elapsed * 1000, 2))
    results_timings['mean total time'] = np.mean(results_timings['tile times'])

In [ ]:
pd.DataFrame.from_dict(tile_results_df, orient='index')

In [ ]:
image = image_and_assets[0].data_as_image()
imshow(image)

# Profile titiler-xarray

In [9]:
# useful to always reload the module while its being developed
%load_ext autoreload
%autoreload
import xarray_tile_reader
import zarr_reader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# bucket = 'nasa-eodc-data-store'
bucket = "carbonplan-benchmarks"
shared_dir = "data/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn"
# chunk_set_paths = ['600_1440_1', '600_1440_29', '365_262_262']
chunk_set_paths = ["data-0", "data-1", "pyramids-v2-3857-0", "pyramids-v2-3857-1", "pyramids-v2-3857-2", "pyramids-v2-3857-3"]

# Create a table with the different chunk shapes and sizes

In [13]:
# datastores = { 
#     'kerchunk': {
#         "data_store_path": f"combined_CMIP6_{temporal_resolution}_{model}_{variable}_kerchunk.json"
#     },
#     'pyramid': {
#         "data_store_path": f"pyramid/CMIP6_{temporal_resolution}_{model}_{variable}.zarr"
#     }
# }

# for chunk_set_path in chunk_set_paths:
#     datastores[chunk_set_path] = {
#         'data_store_path': f"{chunk_set_path}/CMIP6_{temporal_resolution}_{model}_{variable}.zarr"
#     }
datastores = {}
variable = "tasmax"
for chunk_set_path in chunk_set_paths:
    datastores[chunk_set_path] = {
        'data_store_path': f"{shared_dir}/{chunk_set_path}",
    }

for key, datastore in datastores.items():
    reference, anon = False, False
    if key == 'kerchunk':
        reference = True
    if 'pyramid' in key:
        continue # skip for now
    if bucket == 'carbonplan-benchmarks':
        anon = True
    
    data_store_url = f"s3://{bucket}/{datastore['data_store_path']}"

    ds = zarr_reader.xarray_open_dataset(data_store_url, anon=anon, reference=reference)

    var = ds[0][variable]
    chunks = var.encoding.get("chunks", "N/A")
    dtype = var.encoding.get("dtype", "N/A")
    chunks_dict = dict(zip(var.dims, chunks))
    chunk_size_mb = "N/A" if chunks is None else (np.prod(chunks) * dtype.itemsize)/1024/1024    
    datastores[key]['chunk_size_mb'] = chunk_size_mb    
    datastores[key]['chunks'] = chunks_dict
    datastores[key]['dtype'] = dtype

Group is None
Group is None


In [14]:
df = pd.DataFrame.from_dict(datastores, orient='index')
df

,data_store_path,chunk_size_mb,chunks,dtype
data-0,data/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1...,95.581055,"{'time': 29, 'lat': 600, 'lon': 1440}",float32
data-1,data/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1...,3.295898,"{'time': 1, 'lat': 600, 'lon': 1440}",float32
pyramids-v2-3857-0,data/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1...,NaN,NaN,NaN
pyramids-v2-3857-1,data/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1...,NaN,NaN,NaN
pyramids-v2-3857-2,data/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1...,NaN,NaN,NaN
pyramids-v2-3857-3,data/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1...,NaN,NaN,NaN


In [15]:
niters = 1
tile_results_df = {}
dataset_chunk_sizes = {}
results_df = datastores.copy()

for xyz_tile in xyz_tiles:
    results_df[xyz_tile] = {}
    for dataset in datastores.keys():
        data_store_path = datastores[dataset]['data_store_path']
        data_store_url = f"s3://{bucket}/{data_store_path}"
        print(data_store_url)
        reference, group = False, None
        if dataset == 'kerchunk':
            reference = True
        if 'pyramid' in dataset:
            #group = xyz_tile[2]
            continue

        results_df[xyz_tile][dataset] = {
            'time to open (ms)': [],
            'rio reproject (ms)': [],
            'total time (ms)': []
        }
        timings_results = results_df[xyz_tile][dataset]
        for iter in range(niters):
            with Timer() as t:
                image_and_timings, cprofile = xarray_tile_reader.tile(
                    data_store_url,
                    *xyz_tile,
                    reference=reference,
                    anon=False,
                    variable=variable,
                    group = group
                )
            total_time = round(t.elapsed * 1000, 2)

            timings = image_and_timings[1]
            timings_results['time to open (ms)'].append(timings['time_to_open']),
            timings_results['rio reproject (ms)'].append(timings['rio.reproject']),
            timings_results['total time (ms)'].append(total_time)
        timings_results['mean time to open (ms)'] = np.mean(timings_results['time to open (ms)'])
        timings_results['mean rio reproject (ms)'] = np.mean(timings_results['rio reproject (ms)']) 
        timings_results['mean total time (ms)'] = np.mean(timings_results['total time (ms)'])

s3://carbonplan-benchmarks/data/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn/data-0
Group is None
s3://carbonplan-benchmarks/data/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn/data-1
Group is None
s3://carbonplan-benchmarks/data/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn/pyramids-v2-3857-0
s3://carbonplan-benchmarks/data/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn/pyramids-v2-3857-1
s3://carbonplan-benchmarks/data/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn/pyramids-v2-3857-2
s3://carbonplan-benchmarks/data/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn/pyramids-v2-3857-3
s3://carbonplan-benchmarks/data/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS

In [16]:
for xyz_tile in xyz_tiles:
    print(f"Results for {xyz_tile}")
    df = pd.DataFrame.from_dict(results_df[xyz_tile], orient='index')
    print(df.sort_values('mean total time (ms)'))
    print()

Results for (0, 0, 0)
       time to open (ms) rio reproject (ms) total time (ms)  \
data-0          [192.52]           [1015.1]       [1544.06]   
data-1         [1522.57]           [161.17]       [1752.12]   

        mean time to open (ms)  mean rio reproject (ms)  mean total time (ms)  
data-0                  192.52                  1015.10               1544.06  
data-1                 1522.57                   161.17               1752.12  

Results for (0, 0, 1)
       time to open (ms) rio reproject (ms) total time (ms)  \
data-0          [157.43]           [551.87]        [773.05]   
data-1          [1882.9]            [80.95]       [2032.88]   

        mean time to open (ms)  mean rio reproject (ms)  mean total time (ms)  
data-0                  157.43                   551.87                773.05  
data-1                 1882.90                    80.95               2032.88  

Results for (1, 1, 2)
       time to open (ms) rio reproject (ms) total time (ms)  \
data-0   